In [1]:
# Importing libraries and modules

import geopandas as gpd
from geopandas.tools import sjoin
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from shapely import wkt
from scipy.spatial import cKDTree
from shapely.geometry import Point
from shapely.ops import nearest_points

In [2]:
# Reading parking spaces geojson file
df_parking = gpd.read_file("F:/Berlin-DOP_2020/Processed images/Split_images_3/Fixed Predictions.geojson", crs='EPSG:25833')

C:\Users\Bashi\.conda\envs\test\lib\site-packages\geopandas\array.py:91: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [3]:
df_parking

,id,label,geometry
0,1,PK-space-occupied,"MULTIPOLYGON (((391008.088 5821088.357, 391002..."
1,2,PK-space-occupied,"MULTIPOLYGON (((391014.989 5821079.765, 391016..."
2,3,PK-space-occupied,"MULTIPOLYGON (((391025.083 5821069.202, 391022..."
3,4,PK-space-occupied,"MULTIPOLYGON (((391030.060 5821058.873, 391031..."
4,5,PK-space-occupied,"MULTIPOLYGON (((391035.834 5821051.408, 391037..."
...,...,...,...
3858,3861,PK-space-occupied,"MULTIPOLYGON (((391971.400 5824865.150, 391971..."
3859,3862,PK-space-occupied,"MULTIPOLYGON (((391962.000 5824856.250, 391962..."
3860,3863,PK-space-occupied,"MULTIPOLYGON (((391953.700 5824873.350, 391954..."
3861,3864,PK-space-occupied,"MULTIPOLYGON (((391950.000 5824955.050, 391950..."


In [4]:
# Adding the polygon geometries to a seperate column
df_parking['parking_polygon'] = df_parking['geometry']

# Adding the area of parking to a seperate column
df_parking["parking_area_m2"] = df_parking['geometry'].area

# Setting the center point of each parking as the geometry column
df_parking.geometry = df_parking.representative_point()

# Converting the parking polygons to objects
df_parking['parking_polygon'] = df_parking['parking_polygon'].astype(str)

C:\Users\Bashi\.conda\envs\test\lib\site-packages\geopandas\base.py:95: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = getattr(a_this, op)(*args, **kwargs).data
C:\Users\Bashi\.conda\envs\test\lib\site-packages\geopandas\geoseries.py:133: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


In [5]:
df_parking

,id,label,geometry,parking_polygon,parking_area_m2
0,1,PK-space-occupied,POINT (391006.228 5821092.465),MULTIPOLYGON (((391008.0877934272 5821088.3568...,20.828098
1,2,PK-space-occupied,POINT (391012.923 5821084.014),MULTIPOLYGON (((391014.9892018779 5821079.7652...,17.860213
2,3,PK-space-occupied,POINT (391023.295 5821069.883),MULTIPOLYGON (((391025.0830985915 5821069.2018...,6.344641
3,4,PK-space-occupied,POINT (391029.615 5821060.939),MULTIPOLYGON (((391030.0596244131 5821058.8732...,7.703498
4,5,PK-space-occupied,POINT (391035.344 5821053.662),MULTIPOLYGON (((391035.8342723005 5821051.4084...,9.316934
...,...,...,...,...,...
3858,3861,PK-space-occupied,POINT (391971.414 5824867.700),"MULTIPOLYGON (((391971.4 5824865.15, 391971.65...",9.610000
3859,3862,PK-space-occupied,POINT (391959.259 5824860.850),"MULTIPOLYGON (((391962 5824856.25, 391962.4 58...",43.225000
3860,3863,PK-space-occupied,POINT (391953.716 5824874.525),"MULTIPOLYGON (((391953.7 5824873.35, 391954.4 ...",7.310000
3861,3864,PK-space-occupied,POINT (391957.039 5824961.750),"MULTIPOLYGON (((391950 5824955.05, 391950.05 5...",49.940000


In [6]:
# Copying the parking dataframe to merge later
df_parking_copy = df_parking.copy()

# Adding two new columns to add nearest parking space geometry
#df_parking.insert(5, 'nearest_geometry_index', None)
df_parking.insert(5, 'nearest_geometry', None)

In [8]:
# Finding the nearest parking space
for index, row in df_parking.iterrows():
    point = row.geometry
    multipoint = df_parking.drop(index, axis=0).geometry.unary_union
    queried_geom, nearest_geom = nearest_points(point, multipoint)
    df_parking.loc[index, 'nearest_geometry'] = nearest_geom

C:\Users\Bashi\.conda\envs\test\lib\site-packages\pandas\core\internals\managers.py:427: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


In [10]:
# Setting the nerest geometry ad dataframe's geometry
df_parking = df_parking.set_geometry("nearest_geometry")
df_parking['geometry'] = df_parking['geometry'].astype(str)

C:\Users\Bashi\.conda\envs\test\lib\site-packages\geopandas\geodataframe.py:35: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)


In [11]:
df_parking

,id,label,geometry,parking_polygon,parking_area_m2,nearest_geometry
0,1,PK-space-occupied,POINT (391006.2280902515 5821092.464788733),MULTIPOLYGON (((391008.0877934272 5821088.3568...,20.828098,POINT (391012.923 5821084.014)
1,2,PK-space-occupied,POINT (391012.9228625686 5821084.014084507),MULTIPOLYGON (((391014.9892018779 5821079.7652...,17.860213,POINT (391006.228 5821092.465)
2,3,PK-space-occupied,POINT (391023.2948266955 5821069.882629108),MULTIPOLYGON (((391025.0830985915 5821069.2018...,6.344641,POINT (391013.398 5821069.859)
3,4,PK-space-occupied,POINT (391029.6152722856 5821060.938967137),MULTIPOLYGON (((391030.0596244131 5821058.8732...,7.703498,POINT (391022.106 5821058.404)
4,5,PK-space-occupied,POINT (391035.3440769893 5821053.661971832),MULTIPOLYGON (((391035.8342723005 5821051.4084...,9.316934,POINT (391029.615 5821060.939)
...,...,...,...,...,...,...
3858,3861,PK-space-occupied,POINT (391971.4139423077 5824867.7),"MULTIPOLYGON (((391971.4 5824865.15, 391971.65...",9.610000,POINT (391972.812 5824866.550)
3859,3862,PK-space-occupied,POINT (391959.2588235297 5824860.85),"MULTIPOLYGON (((391962 5824856.25, 391962.4 58...",43.225000,POINT (391957.359 5824862.875)
3860,3863,PK-space-occupied,POINT (391953.7161764707 5824874.525),"MULTIPOLYGON (((391953.7 5824873.35, 391954.4 ...",7.310000,POINT (391954.056 5824866.900)
3861,3864,PK-space-occupied,POINT (391957.0386363636 5824961.75),"MULTIPOLYGON (((391950 5824955.05, 391950.05 5...",49.940000,POINT (391948.775 5824954.600)


In [12]:
# Calculating the distance from each parking space to the nearest parking space
df_parking["nearest_geometry_dis"] = df_parking.distance(df_parking_copy)

In [13]:
# Adding the other data columns
df_merge = df_parking_copy.loc[:, ["id","geometry","parking_area_m2"]]
df_merge = df_merge.rename(columns={'geometry': 'nearest_geometry', 'parking_area_m2': 'parking_area_nearest_m2'})

In [14]:
# Joining the final dataframes
Merged_df = df_parking.merge(df_merge, on='nearest_geometry', how='inner')

In [15]:
Merged_df

,id_x,label,geometry,parking_polygon,parking_area_m2,nearest_geometry,nearest_geometry_dis,id_y,parking_area_nearest_m2
0,1,PK-space-occupied,POINT (391006.2280902515 5821092.464788733),MULTIPOLYGON (((391008.0877934272 5821088.3568...,20.828098,POINT (391012.923 5821084.014),10.781205,2,17.860213
1,2,PK-space-occupied,POINT (391012.9228625686 5821084.014084507),MULTIPOLYGON (((391014.9892018779 5821079.7652...,17.860213,POINT (391006.228 5821092.465),10.781205,1,20.828098
2,3,PK-space-occupied,POINT (391023.2948266955 5821069.882629108),MULTIPOLYGON (((391025.0830985915 5821069.2018...,6.344641,POINT (391013.398 5821069.859),9.897025,7,44.395953
3,4,PK-space-occupied,POINT (391029.6152722856 5821060.938967137),MULTIPOLYGON (((391030.0596244131 5821058.8732...,7.703498,POINT (391022.106 5821058.404),7.925644,6,19.424056
4,5,PK-space-occupied,POINT (391035.3440769893 5821053.661971832),MULTIPOLYGON (((391035.8342723005 5821051.4084...,9.316934,POINT (391029.615 5821060.939),9.261418,4,7.703498
...,...,...,...,...,...,...,...,...,...
3858,3862,PK-space-occupied,POINT (391959.2588235297 5824860.85),"MULTIPOLYGON (((391962 5824856.25, 391962.4 58...",43.225000,POINT (391957.359 5824862.875),2.776756,3858,69.010000
3859,3860,PK-space-occupied,POINT (391972.8125 5824866.55),"MULTIPOLYGON (((391977 5824862.65, 391977.8 58...",27.605000,POINT (391971.414 5824867.700),1.810653,3861,9.610000
3860,3861,PK-space-occupied,POINT (391971.4139423077 5824867.7),"MULTIPOLYGON (((391971.4 5824865.15, 391971.65...",9.610000,POINT (391972.812 5824866.550),1.810653,3860,27.605000
3861,3863,PK-space-occupied,POINT (391953.7161764707 5824874.525),"MULTIPOLYGON (((391953.7 5824873.35, 391954.4 ...",7.310000,POINT (391954.056 5824866.900),7.632571,3859,8.245000


In [16]:
# Setting the geometry to the parking space polygons
Merged_df['geometry'] = Merged_df['geometry'].astype(str)
Merged_df['nearest_geometry'] = Merged_df['nearest_geometry'].astype(str)

# Converting to a geo dataframe
Merged_df['parking_polygon'] = Merged_df['parking_polygon'].apply(wkt.loads)
Merged_df = gpd.GeoDataFrame(Merged_df, geometry="parking_polygon", crs='EPSG:25833')

C:\Users\Bashi\.conda\envs\test\lib\site-packages\geopandas\array.py:91: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [17]:
Merged_df.dtypes

id_x                          int64
label                        object
geometry                     object
parking_polygon            geometry
parking_area_m2             float64
nearest_geometry             object
nearest_geometry_dis        float64
id_y                          int64
parking_area_nearest_m2     float64
dtype: object

In [18]:
Merged_df.to_file("F:/Berlin-DOP_2020/Processed images/Split_images_3/parking_dist.gpkg", driver="GPKG", layer="parking_dist")

In [19]:
Merged_df

,id_x,label,geometry,parking_polygon,parking_area_m2,nearest_geometry,nearest_geometry_dis,id_y,parking_area_nearest_m2
0,1,PK-space-occupied,POINT (391006.2280902515 5821092.464788733),"MULTIPOLYGON (((391008.088 5821088.357, 391002...",20.828098,POINT (391012.9228625686 5821084.014084507),10.781205,2,17.860213
1,2,PK-space-occupied,POINT (391012.9228625686 5821084.014084507),"MULTIPOLYGON (((391014.989 5821079.765, 391016...",17.860213,POINT (391006.2280902515 5821092.464788733),10.781205,1,20.828098
2,3,PK-space-occupied,POINT (391023.2948266955 5821069.882629108),"MULTIPOLYGON (((391025.083 5821069.202, 391022...",6.344641,POINT (391013.3978297458 5821069.859154929),9.897025,7,44.395953
3,4,PK-space-occupied,POINT (391029.6152722856 5821060.938967137),"MULTIPOLYGON (((391030.060 5821058.873, 391031...",7.703498,POINT (391022.1060427084 5821058.403755868),7.925644,6,19.424056
4,5,PK-space-occupied,POINT (391035.3440769893 5821053.661971832),"MULTIPOLYGON (((391035.834 5821051.408, 391037...",9.316934,POINT (391029.6152722856 5821060.938967137),9.261418,4,7.703498
...,...,...,...,...,...,...,...,...,...
3858,3862,PK-space-occupied,POINT (391959.2588235297 5824860.85),"MULTIPOLYGON (((391962.000 5824856.250, 391962...",43.225000,POINT (391957.3588900862 5824862.875),2.776756,3858,69.010000
3859,3860,PK-space-occupied,POINT (391972.8125 5824866.55),"MULTIPOLYGON (((391977.000 5824862.650, 391977...",27.605000,POINT (391971.4139423077 5824867.7),1.810653,3861,9.610000
3860,3861,PK-space-occupied,POINT (391971.4139423077 5824867.7),"MULTIPOLYGON (((391971.400 5824865.150, 391971...",9.610000,POINT (391972.8125 5824866.55),1.810653,3860,27.605000
3861,3863,PK-space-occupied,POINT (391953.7161764707 5824874.525),"MULTIPOLYGON (((391953.700 5824873.350, 391954...",7.310000,POINT (391954.0560606061 5824866.9),7.632571,3859,8.245000


In [20]:
# def ckdnearest(gdA, gdB):
#     assert len(gdA) > 0
#     assert len(gdB) > 0
#     nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
#     nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
#     btree = cKDTree(nB)
#     dist, idx = btree.query(nA, k=1)
#     gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
#     gdf = pd.concat(
#         [gdA.reset_index(drop=True), gdB_nearest, pd.Series(dist, name="dist")],
#         axis=1,
#     )

#     return gdf

In [21]:
# def catcherror(x):
#     try:
#         return x.x
#     except:
#         return np.nan

In [22]:
# df_parking.loc[df_parking.geometry.apply(lambda x: catcherror(x)).isna(),:]

In [23]:
# ckdnearest(df_parking, df_parking_copy)

In [24]:
# from scipy import spatial

# # get list of points
# points = df_parking['geometry'].apply(
#                 lambda g:[g.x,g.y]).tolist()
# #spatially organising the points on a tree for quick nearest neighbors calc
# kdtree = spatial.KDTree(points)

# #calculates the nearest neighbors of each point
# _ , neighs = kdtree.query(points, k=2)

# # remove itself as neighbor
# neighs = neighs[:,1]


# # add column to df
# df_parking['nearest_points'] = df_parking.iloc[neighs, 3].tolist()